# Lab 6: Convolutional Network Architectures
## by Michael Doherty, Leilani Guzman, and Carson Pittman

Link to the dataset: https://www.kaggle.com/datasets/brsdincer/vehicle-detection-image-set

## 1. Preparation
### 1.1 Performance Metric

### 1.2 Training and Testing Method

## 2. Modeling
### 2.1 Data Augmentation

### 2.2 Model Creation

### 2.3 Model Visualization and Comparison

### 2.4 Convolutional Network vs. Multi-Layer Perceptron

## 3. Transfer Learning
### 3.1 Transfer Learning Model

### 3.2 Model Comparison